# Programmeertaal C

In deze paragraaf voegen we de programmeertaal C toe aan TESTed.

## Ondersteunde functionaliteit

Voor we van start gaan, overlopen we eerst kort wat we willen ondersteunen.
Uiteraard is dit zoveel mogelijk, maar vooral op het vlak van gegevenstypes moeten we soms keuzes maken.

Welke basistypes gaan we niet ondersteunen?

- `sequence` - Arrays zijn een speciaal geval in C: statische arrays kunnen bijvoorbeeld niet als returnwaarde dienen, en ook als functieargument zijn ze niet ideaal. Dynamische arrays nemen de vorm aan van een pointer en een grootte. Doordat dit twee waarden zijn, ondersteunt TESTed dat momenteel niet.
- `set` - C heeft geen ingebouwde verzamelingen.
- `map` - C heeft geen ingebouwde map of dict. Er zijn wel structs, maar daarvan is het niet mogelijk om at runtime de velden te achterhalen, wat ze niet bruikbaar maakt.

Welke geavanceerde types gaan we niet ondersteunen?

- `big_int` - C heeft geen ingebouwd type voor getallen van arbitraire grootte.
- `fixex_precision`- C heeft geen ingebouwd type voor kommagetallen me arbitraire precisie.
- Andere datastructuren, zoals `array` en `list` (voor de redenen van hierboven). Ook `tuple` wordt niet ondersteund, omdat het niet bestaat in C.

## Configuratieklasse

De eerste stap in het toevoegen van een programmeertaal is het implementeren van de configuratieklasse.
Maak hiervoor een nieuw Python-bestand in `test/langauges/c/config.py`.
Hierin moet een klasse komen die `Language` uitbreidt:

In [ ]:
from tested.languages import Language

class CConfig(Language):
    pass

De klasse `Language` heeft veel functies, waarvan een deel verplicht te implementeren is.
Hieronder overlopen we de functies per onderwerp.

### Programmeertaaleigenschappen

Als eerste zijn er enkele functies die weinig uitleg behoeven, en enkele eigenschappen van C implementeren.


In [ ]:
def file_extension(self) -> str:
        return "c"

De bestandsextensie voor bestanden in C. Er zijn in C ook headers (`.h`), maar daar moeten we geen rekening mee houden.


In [ ]:
def submission_name(self, plan: Plan) -> str:
    return "submission"

De naam van het bestand met de oplossing van de student.

In [ ]:
def selector_name(self) -> str:
    return "selector"

def needs_selector(self):
    return True

De naam van de selector voor batchcompilatie en of dat een selector nodig is (in programmeertalen zoals Python is dat niet nodig).

In [ ]:
def context_name(self, tab_number: int, context_number: int) -> str:
    return f"context_{tab_number}_{context_number}"

De naam van een bestand voor de context van een specifiek tabblad.

In [ ]:
def conventionalise_function(self, function_name: str) -> str:
    return decamelize(function_name)

Deze functie zet namen uit het testplan om naar de namen volgens de gebruiken van de taal. In C kiezen we ervoor om functies in `snake_case` te zetten.

### Compileren en uitvoeren van code

De eerste twee functies die we gaan bekijken zorgen ervoor dat de code die TESTed gegenereerd heeft, gecompileerd en uitgevoerd kan worden.

Als eerste bekijken we het compileren:

In [ ]:
def generation_callback(self, files: List[str]) -> CallbackResult:
        main_file = files[-1]
        exec_file = Path(main_file).stem
        result = executable_name(exec_file)
        return ["gcc", "-std=c11", "-Wall", main_file, "-o", result], [result]

Zoals de naam doet vermoeden wordt deze functie opgeroepen nadat TESTed de code heeft gegenereerd.
Als argument wordt een lijst van bestanden meegegeven die gecompileerd moeten worden tot één uitvoerbaar geheel.
Per conventie is het laatste bestand in de lijst het bestand met de `main`-functie.

Het eerste dat we doen is het bepalen van de naam van de executable.
De functie `executable_name` is een hulpfunctie die de juiste extensie aan een string plakt, afhankelijk van het platform (niets op Linux, `exe` op Windows).

Als returnwaarde verwacht deze functie een tuple met twee elementen:

- Het uit te voeren commando voor de compilatie. In dit geval gebruiken we GCC. Als opties geven we `std=c11` en `Wall` mee. Dit is uiteraard te kiezen per programmeertaal.
- Een lijst van resulterende bestanden. Bij C is er maar één bestand: de executable.

Een concreet voorbeeld maakt dit duidelijker. Stel dat we een oefening met één context compileren in batchcompilatie:

In [ ]:
>>> generation_callback(["submission.c", "context0.c", "selector.c"])
(["gcc", "-std=c11", "-Wall", "selector.c", "-o", "selector.exe"], ["selector.exe"])

Nu de code gecompileerd is, moeten we hem nog steeds kunnen uitvoeren. Dit gebeurt met een gelijkaardige functie:


In [ ]:
def execution_command(self,
                      cwd: Path,
                      file: str,
                      arguments: List[str]) -> List[str]:
    local_file = cwd / executable_name(Path(file).stem)
    return [str(local_file.absolute()), *arguments]

Deze functie heeft meer argumenten:

- `cwd` of _current working directory_, de map waar de uitvoering zal plaatsvinden.
- `file`, het bestand dat uitgevoerd moet worden (het bestand met de `main`-functie). In het geval van C zal dat altijd de executable zijn.
- `arguments`, argumenten die aan het programma meegegeven moeten worden.

Aangezien C compileert tot een uitvoerbaar bestand, kunnen we dat bestand hier direct oproepen. Het enige dat we doen is opnieuw de juiste extensie toevoegen en een absoluut pad gebruiken. Dat laatst omdat het commando door TESTed met `subprocess` uitgevoerd wordt, en dus het pad naar het uitvoerbaar bestand nodig heeft.

De returnwaarde is hier het uit te voeren commando.

### Aanpassen van de oplossingscode

In TESTed kunnen er meerdere `main`-functies zijn:

- De oplossing van de student kan een `main`-functie hebben.
- Zowel de contexten als de selector kunnen `main`-functies hebben.

C staat slechts één `main`-functie toe.

Een ander probleem is dat de selector elke context include (zoals we later zullen zien bij de sjablonen), en elke context ook de oplossing include.

Om deze redenen moeten we de code van de oplossing een beetje aanpassen:

- We voegen aan `guard` toe, zodat de oplossing eenmalig geladen wordt.
- We hernoemen de `main`-functie naar `solution_main` indien die bestaat.

In [ ]:
def solution_callback(self, solution: Path, plan: Plan):
    with open(solution, "r") as file:
        contents = file.read()
    with open(solution, "w") as file:
        header = "#ifdef INCLUDED\n#pragma once\n#endif\n\n"
        result = header + contents.replace("main", "solution_main")
        file.write(result)

### Ondersteunde functionaliteit

Bij de voorbereiding hebben we besproken welke functionaliteit we willen ondersteunen. Hier implementeren we dat:

In [ ]:

def type_support_map(self) -> Mapping[AllTypes, TypeSupport]:
    return fallback(super().type_support_map(), {
        ant.INT_8:            TypeSupport.SUPPORTED,
        ant.U_INT_8:          TypeSupport.SUPPORTED,
        ant.INT_16:           TypeSupport.SUPPORTED,
        ant.U_INT_16:         TypeSupport.SUPPORTED,
        ant.INT_32:           TypeSupport.SUPPORTED,
        ant.U_INT_32:         TypeSupport.SUPPORTED,
        ant.INT_64:           TypeSupport.SUPPORTED,
        ant.U_INT_64:         TypeSupport.SUPPORTED,
        ant.SINGLE_PRECISION: TypeSupport.SUPPORTED,
        ant.DOUBLE_EXTENDED:  TypeSupport.SUPPORTED,
        ant.FIXED_PRECISION:  TypeSupport.UNSUPPORTED,
        ast.ARRAY:            TypeSupport.SUPPORTED,
        ast.LIST:             TypeSupport.UNSUPPORTED,
        ast.TUPLE:            TypeSupport.UNSUPPORTED,
        bst.SET:              TypeSupport.UNSUPPORTED,
        bot.MAP:              TypeSupport.UNSUPPORTED,
    })

def supported_constructs(self) -> Constructs:
    return (Constructs.MAIN | Constructs.NOTHING | Constructs.FUNCTION_CALL
            | Constructs.ASSIGNMENT)

Eerst sommen we de ondersteunde en niet-ondersteunde types op. In de tweede functie geven we aan welke _constructs_ we ondersteunen.

## Sjablonen

De tweede stap is het implementeren van de sjablonen.
